### Testing for Interactive use case

In [13]:
# pip install --upgrade ray-on-aml

In [6]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from IPython.display import clear_output
import time
import platform
import sys
# sys.path.append("../") # go to parent dir
import importlib


In [17]:
#You can pre-provision "worker-cpu-v3" in the same vnet with your compute instance
from ray_on_aml.core import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3")
_, ray = ray_on_aml.getRay()


In [9]:
ray.cluster_resources()

{'node:10.0.0.26': 1.0,
 'CPU': 24.0,
 'object_store_memory': 24947750089.0,
 'memory': 56963684356.0,
 'node:10.0.0.18': 1.0,
 'node:10.0.0.17': 1.0,
 'node:10.0.0.21': 1.0,
 'node:10.0.0.20': 1.0,
 'node:10.0.0.25': 1.0}

In [17]:
# ray_on_aml.shutdown()


### Testing with Dask on Ray

In [10]:
# import ray
# ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
dask.config.set(scheduler=ray_dask_get)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

,grade
age,
8,8738.0
22,5914.5
27,6655.0
42,2169.0
45,4395.0
...,...
9952,2387.0
9973,2121.0
9984,779.0


In [13]:
# import dask.dataframe as dd

# storage_options = {'account_name': 'azureopendatastorage'}
# ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
# ddf.count().compute()
#This still have error about parquet, need to fix, might be lib version conflict

In [11]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
from adlfs import AzureBlobFileSystem
account_key = ws.get_default_keyvault().get_secret("adls7-account-key")
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)

data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs2)
data2 = ray.data.read_parquet("az://mltraining/ISDWeatherDelta/year2008", filesystem=abfs)
data.count()

Metadata Fetch Progress: 100%|██████████| 8/8 [00:00<00:00, 10.52it/s]


98904376

### Testing Ray Tune for distributed ML tunning

In [12]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


2021-12-10 22:29:33,459	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-10 22:29:33,475	INFO logger.py:605 -- pip install "ray[tune]" to see TensorBoard files.
2021-12-10 22:29:33,476	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,lr,momentum
train_mnist_a61c4_00000,PENDING,,6.14092e-10,0.0582547


2021-12-10 22:29:34,003	ERROR trial_runner.py:958 -- Trial train_mnist_a61c4_00000: Error processing event.
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 924, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/tune/ray_trial_executor.py", line 787, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/worker.py", line 1715, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::ImplicitFunc.__init__() (pid=207, ip=10.0.0.25)
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. T

Result for train_mnist_a61c4_00000:
  trial_id: a61c4_00000
  


(TemporaryActor pid=207, ip=10.0.0.25) 2021-12-10 22:29:33,999	ERROR worker.py:431 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::ImplicitFunc.__init__() (pid=207, ip=10.0.0.25)
(TemporaryActor pid=207, ip=10.0.0.25) RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:
(TemporaryActor pid=207, ip=10.0.0.25) 
(TemporaryActor pid=207, ip=10.0.0.25) ray::ImplicitFunc.__init__() (pid=207, ip=10.0.0.25)
(TemporaryActor pid=207, ip=10.0.0.25)   File "/azureml-envs/azureml_f868bc5adb64a16ac9e789bb1315cb41/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle.py", line 679, in subimport
(TemporaryActor pid=207, ip=10.0.0.25)     __import__(name)
(TemporaryActor pid=207, ip=10.0.0.25)   File "/azureml-envs/azureml_f868bc5adb64a16ac9e789bb1315cb41/lib/python3.8/site-packages/torchvision/__init__.py", line 6, in <modul

TuneError: ('Trials did not complete', [train_mnist_a61c4_00000])

In [10]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)


2021-12-10 21:43:15,768	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_2e77d_00000,RUNNING,10.0.0.26:23057,0.0116743,2,3,0.564379
train_breast_cancer_2e77d_00001,PENDING,,0.00371402,8,2,0.72637
train_breast_cancer_2e77d_00002,PENDING,,0.000417622,7,3,0.830038
train_breast_cancer_2e77d_00003,PENDING,,0.0131403,4,3,0.854627
train_breast_cancer_2e77d_00004,PENDING,,0.00178065,1,2,0.944164
train_breast_cancer_2e77d_00005,PENDING,,0.0124743,7,1,0.805848
train_breast_cancer_2e77d_00006,PENDING,,0.00551973,8,2,0.586203
train_breast_cancer_2e77d_00007,PENDING,,0.00997637,6,2,0.712431
train_breast_cancer_2e77d_00008,PENDING,,0.000510061,1,3,0.733943
train_breast_cancer_2e77d_00009,PENDING,,0.0695749,6,3,0.935206


Result for train_breast_cancer_2e77d_00007:
  date: 2021-12-10_21-43-17
  done: true
  experiment_id: e46e5e5eecba438cb39b79b7d7950442
  hostname: c2473fa89b9a40488624f53530886d6100000C
  iterations_since_restore: 1
  mean_accuracy: 0.909091
  node_ip: 10.0.0.25
  pid: 195
  time_since_restore: 0.10215330123901367
  time_this_iter_s: 0.10215330123901367
  time_total_s: 0.10215330123901367
  timestamp: 1639172597
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2e77d_00007
  
Result for train_breast_cancer_2e77d_00002:
  date: 2021-12-10_21-43-17
  done: true
  experiment_id: a6d3135d2f364b7ab8b28612c3b5e35d
  hostname: c2473fa89b9a40488624f53530886d6100000A
  iterations_since_restore: 1
  mean_accuracy: 0.93007
  node_ip: 10.0.0.20
  pid: 210
  time_since_restore: 0.09505844116210938
  time_this_iter_s: 0.09505844116210938
  time_total_s: 0.09505844116210938
  timestamp: 1639172597
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2e77d_00002
  
Result fo

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_2e77d_00000,TERMINATED,10.0.0.26:23057,0.0116743,2,3,0.564379,0.958042,1,1.04478
train_breast_cancer_2e77d_00001,TERMINATED,10.0.0.26:23111,0.00371402,8,2,0.72637,0.93007,1,0.291249
train_breast_cancer_2e77d_00002,TERMINATED,10.0.0.20:210,0.000417622,7,3,0.830038,0.93007,1,0.0950584
train_breast_cancer_2e77d_00003,TERMINATED,10.0.0.18:196,0.0131403,4,3,0.854627,0.93007,1,0.168231
train_breast_cancer_2e77d_00004,TERMINATED,10.0.0.17:195,0.00178065,1,2,0.944164,0.909091,1,0.119618
train_breast_cancer_2e77d_00005,TERMINATED,10.0.0.17:197,0.0124743,7,1,0.805848,0.923077,1,0.104501
train_breast_cancer_2e77d_00006,TERMINATED,10.0.0.12:230,0.00551973,8,2,0.586203,0.909091,1,0.154502
train_breast_cancer_2e77d_00007,TERMINATED,10.0.0.25:195,0.00997637,6,2,0.712431,0.909091,1,0.102153
train_breast_cancer_2e77d_00008,TERMINATED,10.0.0.18:197,0.000510061,1,3,0.733943,0.916084,1,0.138657
train_breast_cancer_2e77d_00009,TERMINATED,10.0.0.12:232,0.0695749,6,3,0.935206,0.902098,1,0.139962


2021-12-10 21:43:18,123	INFO tune.py:626 -- Total run time: 2.36 seconds (2.21 seconds for the tuning loop).


### Testing Spark on Ray (#todo)

In [15]:
# import ray
# import raydp
# import os
# ray.shutdown()
# ray.init()
# os.environ["PYSPARK_PYTHON"]="/anaconda/envs/azureml_py38/bin/python3"

# # ray.init(address ='ray://10.0.0.11:6379')
# spark = raydp.init_spark(
#   app_name = "example",
#   num_executors = 2,
#   executor_cores = 1,
#   executor_memory = "1gb"
# )

# # data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")


# # # normal data processesing with Spark
# # df = spark.createDataFrame([('look',), ('spark',), ('tutorial',), ('spark',), ('look', ), ('python', )], ['word'])
# # df.show()
# # word_count = df.groupBy('word').count()
# # word_count.show()
# import pandas as pd

# from pyspark.sql.functions import col, pandas_udf
# from pyspark.sql.types import LongType

# # Declare the function and create the UDF
# def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
#     return a * b

# multiply = pandas_udf(multiply_func, returnType=LongType())

# # The function for a pandas_udf should be able to execute with local Pandas data
# x = pd.Series([1, 2, 3])
# print(multiply_func(x, x))
# # 0    1
# # 1    4
# # 2    9
# # dtype: int64

# # Create a Spark DataFrame, 'spark' is an existing SparkSession
# df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# # Execute function as a Spark vectorized UDF
# df.select(multiply(col("x"), col("x"))).show()
# # +-------------------+
# # |multiply_func(x, x)|
# # +-------------------+
# # |                  1|
# # |                  4|
# # |                  9|
# # +-------------------+


# # stop the spark cluster
# raydp.stop_spark()


## Testing Ray on Job Cluster

In [14]:
ws = Workspace.from_config()

# base_conda_dep =['adlfs>=2021.10.0','pytorch','matplotlib','torchvision','pip']
# base_pip_dep = ['sklearn','xgboost','lightgbm','ray[default]==1.9.0', 'xgboost_ray', 'dask','pyarrow>=6.0.1', 'azureml-mlflow']
compute_cluster = 'worker-cpu-v3'
maxnode =5
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='ray_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)


# python_version = ["python="+platform.python_version()]



# conda_packages = python_version+base_conda_dep
# pip_packages = base_pip_dep 

# conda_dep = CondaDependencies()

# rayEnv = Environment(name="rayEnv")
rayEnv = Environment.get(ws, "rayEnv", version=18)
# for conda_package in conda_packages:
#     conda_dep.add_conda_package(conda_package)

# for pip_package in pip_packages:
#     conda_dep.add_pip_package(pip_package)

# # Adds dependencies to PythonSection of myenv
# rayEnv.python.conda_dependencies=conda_dep

src = ScriptRunConfig(source_directory='job',
                script='aml_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode),
                    # arguments = ["--master_ip",master_ip]
                )
run = Experiment(ws, exp).submit(src)

Found existing cluster, use it.
